In [4]:
import pandas as pd
import numpy as np

orders = pd.read_csv("../data/raw/olist_orders_dataset.csv")
items = pd.read_csv("../data/raw/olist_order_items_dataset.csv")
payments = pd.read_csv("../data/raw/olist_order_payments_dataset.csv")
reviews = pd.read_csv("../data/raw/olist_order_reviews_dataset.csv")
customers = pd.read_csv("../data/raw/olist_customers_dataset.csv")
products = pd.read_csv("../data/raw/olist_products_dataset.csv")
categories = pd.read_csv("../data/raw/product_category_name_translation.csv")

date_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors="coerce")
    
products = products.merge(
    categories,
    on="product_category_name",
    how="left"
)

df = (
    items
    .merge(orders, on="order_id", how="left")
    .merge(customers, on="customer_id", how="left")
    .merge(products, on="product_id", how="left")
    .merge(payments, on="order_id", how="left")
    .merge(reviews, on="order_id", how="left")
)

df["revenue"] = df["price"] + df["freight_value"]

df["order_month"] = df["order_purchase_timestamp"].dt.to_period("M").astype(str)
df["order_year"] = df["order_purchase_timestamp"].dt.year
df["order_weekday"] = df["order_purchase_timestamp"].dt.day_name()

df = df[df["order_status"] == "delivered"]

df.to_csv("../data/processed/clean_olist_data.csv", index=False)